In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import mne
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import TopKCategoricalAccuracy, AUC, Precision, Recall
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [ ]:
BCICIV_PATH = "../toy_data/BCICIV_2a_gdf"
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = BATCH_SIZE * 2 

EPOCHS = 5
LEARNING_RATE = 1e-3

In [ ]:
def load_data():
	return mne.io.read_raw_gdf(f'{BCICIV_PATH}/A01T.gdf')

def visualize_data():
	pass

def preprocess_data():
	pass

def make_tf_dataset(X_train, y_train, X_valid, y_valid, X_test, y_test):
	train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
	valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
	test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

	train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
	valid_dataset = valid_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
	test_dataset = test_dataset.batch(BATCH_SIZE)

	return train_dataset, valid_dataset, test_dataset

In [ ]:
def get_callbacks():
	return [
		# ModelCheckpoint(
		#     "best_model.h5", save_best_only=True, monitor="loss"
		# ),
		# ReduceLROnPlateau(
		#     monitor="val_top_k_categorical_accuracy",
		#     factor=0.2,
		#     patience=2,
		#     min_lr=0.000001,
		# ),
		EarlyStopping(
			monitor="val_loss",
			min_delta=1e-3,
			patience=10,
			verbose=1,
			mode="auto",
			baseline=None,
			restore_best_weights=True,
		),
	]

def EEGNet(n_classes, channels=64, samples=128, kernel_length=64, n_filters1=8, n_filters2=16, depth_multiplier=2, norm_rate=0.25, dropout_rate=0.5, dropoutType="Dropout"):

	if dropoutType == "SpatialDropout2D":
		dropoutType=SpatialDropout2D
	elif dropoutType == "Dropout":
		dropoutType=Dropout

	input = Input(shape=(channels, samples, 1))

	block1 = Conv2D(n_filters1, (1, kernel_length), padding="same", input_shape=(channels, samples, 1), use_bias=False)(input)
	block1 = BatchNormalization()(block1)
	block1 = DepthwiseConv2D((channels, 1), use_bias=False, depth_multiplier=depth_multiplier, depthwise_constraint=max_norm(1.0))(block1)
	block1 = BatchNormalization()(block1)
	block1 = Activation("elu")(block1)
	block1 = AveragePooling2D((1, 4))(block1)
	block1 = dropoutType(dropout_rate)(block1)

	block2 = SeparableConv2D(n_filters2, (1, 16), use_bias=False, padding="same")(block1)
	block2 = BatchNormalization()(block2)
	block2 = Activation("elu")(block2)
	block2 = AveragePooling2D((1, 8))(block2)
	block2 = dropoutType(dropout_rate)(block2)
	block2 = Flatten(name="flatten")(block2)

	classifier = Dense(n_classes, name="dense", kernel_constraint=max_norm(norm_rate))(block2)
	classifier = Activation("softmax", name="softmax")(classifier)

	model = Model(inputs=input, outputs=classifier) 

	optimizer = Adam(amsgrad=True, learning_rate=LEARNING_RATE)
	loss = CategoricalCrossentropy()

	model.compile(
		optimizer=optimizer,
		loss=loss,
		metrics=[
			TopKCategoricalAccuracy(k=3),
			AUC(),
			Precision(),
			Recall(),
		],
	)

	model.summary()

	return model

In [ ]:
def plot_history_metrics(history):
    total_plots = len(history.history)
    cols = total_plots // 2

    rows = total_plots // cols

    if total_plots % cols != 0:
        rows += 1

    pos = range(1, total_plots + 1)
    plt.figure(figsize=(15, 10))
    for i, (key, value) in enumerate(history.history.items()):
        plt.subplot(rows, cols, pos[i])
        plt.plot(range(len(value)), value)
        plt.title(str(key))
    plt.show()

In [ ]:
data = load_data()
visualize_data(data)
X_train, y_train, X_valid, y_valid, X_test, y_test = preprocess_data(data)
train_dataset, valid_dataset, test_dataset = make_tf_dataset(X_train, y_train, X_valid, y_valid, X_test, y_test)

In [ ]:
model = EEGNet(n_classes=???)

In [ ]:
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    callbacks=get_callbacks(),
    validation_data=valid_dataset,
    class_weight=???,
)

In [ ]:
plot_history_metrics(history)